In [1]:
import pymysql
import numpy as np
import pandas as pd
from typing import List

In [2]:
# connect to mysql
conn = pymysql.connect(host='MacBook-Pro-5.local', user='root', password='zsh558467', database='covid19', charset='utf8')

In [3]:
test = pd.read_sql('select * from test', conn)
death = pd.read_sql('select * from death', conn)
hospital = pd.read_sql('select * from hospital', conn)
vaccination = pd.read_sql('select * from vaccination', conn)
foodstore = pd.read_sql('select * from foodstore', conn)
test.tail(10)

,test_date,county,new_positive,total_positive,new_test,total_test,daily_rate
62238,2022-11-29,Sullivan,23,23315,265,378581,0.09
62239,2022-11-29,Tioga,13,13566,265,234960,0.05
62240,2022-11-29,Tompkins,20,25569,226,3061104,0.09
62241,2022-11-29,Ulster,27,40893,368,869631,0.07
62242,2022-11-29,Warren,14,18074,249,341172,0.07
62243,2022-11-29,Washington,20,15107,223,289104,0.08
62244,2022-11-29,Wayne,10,21075,252,384353,0.04
62245,2022-11-29,Westchester,405,317167,4321,5881655,0.09
62246,2022-11-29,Wyoming,2,9698,45,158509,0.04
62247,2022-11-29,Yates,3,4293,84,91974,0.05


In [4]:
# prepare data for "Top 20 Positive & Vaccination"
sql_tv = 'select t.county, total_positive, first_dose, series_complete from test t, vaccination v where t.county=v.county and t.test_date=v.report_date and test_date="2022-11-29"'
tv = pd.read_sql(sql_tv, conn)
tv = tv.sort_values(by='total_positive', ascending=False)[0:20]
x_county_tv = np.array(tv['county'])
y_posi = np.array(tv['total_positive'])
y_posi = [str(i) for i in list(y_posi)]
y_first = np.array(tv['first_dose'])
y_first = [str(i) for i in list(y_first)]
y_series = np.array(tv['series_complete'])
y_series = [str(i) for i in list(y_series)]

In [5]:
# prepare data for "Top 20 fatality & ICU beds"
sql_dh = 'select d.county, fatality, AVG(staffed_beds), AVG(staffed_ICU_beds) from death d, hospital h where d.county=h.county and d.report_date="2022-11-29" group by h.county'
dh = pd.read_sql(sql_dh, conn)
dh = dh.sort_values(by='fatality', ascending=False)[0:20]
x_county_dh = np.array(dh['county'])
y_fata = np.array(dh['fatality'])
y_fata = [str(i) for i in list(y_fata)]
y_bed = np.array(dh['AVG(staffed_beds)'])
y_bed = [str(i) for i in list(y_bed)]
y_icu_bed = np.array(dh['AVG(staffed_ICU_beds)'])
y_icu_bed = [str(i) for i in list(y_icu_bed)]

In [6]:
# prepare data for "Top 5 Counties' Positive Trend"
monthly_positive = test[['test_date', 'county', 'new_positive']]

def sep_yearmonths(x):
    x['month'] = str(x['test_date'])[:-3]
    return x

monthly_positive = monthly_positive.apply(sep_yearmonths,axis=1)

monthly_positive = monthly_positive.groupby(['county','month'], as_index=False).sum()

x1 = np.array(monthly_positive['month'][monthly_positive.county=="Albany"])
y1 = np.array(monthly_positive['new_positive'][monthly_positive.county=="Kings"])
y1 = [str(i) for i in list(y1)]
y2 = np.array(monthly_positive['new_positive'][monthly_positive.county=="Queens"])
y2 = [str(i) for i in list(y2)]
y3 = np.array(monthly_positive['new_positive'][monthly_positive.county=="New York"])
y3 = [str(i) for i in list(y3)]
y4 = np.array(monthly_positive['new_positive'][monthly_positive.county=="Nassau"])
y4 = [str(i) for i in list(y4)]
y5 = np.array(monthly_positive['new_positive'][monthly_positive.county=="Bronx"])
y5 = [str(i) for i in list(y5)]

In [7]:
# of Foodstores And Positive Rate
sql_ft = 'select t.county, total_positive/total_test, count(license_id) from test t, foodstore f where test_date="2022-11-29" and t.county=f.county group by f.county'
ft = pd.read_sql(sql_ft, conn)
ft = ft.sort_values(by='count(license_id)', ascending=False)
x_county = np.array(ft['county'])
y_posi_rate = np.array(ft['total_positive/total_test'])
y_posi_rate = [str(i) for i in list(y_posi_rate)]
y_fs = np.array(ft['count(license_id)'])
y_fs = [str(i) for i in list(y_fs)]

In [8]:
import pyecharts.options as opts
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
from pyecharts.charts import Bar
from pyecharts.charts import Line
from pyecharts.faker import Faker
from pyecharts.charts import Page

# "Top20 County Positive & Vaccination"
Bar1 = (
    Bar()
        .add_xaxis(list(x_county_tv))
        .add_yaxis("positive", y_posi)
        .extend_axis(
            yaxis=opts.AxisOpts(
                type_='value', name='first&series', is_show=True, min_=0, max_=2500000, position='right'
            )
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="Top20 County Positive & Vaccination"),
                        tooltip_opts=opts.TooltipOpts(trigger='axis',axis_pointer_type='cross'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
Line1 = (
    Line()
        .add_xaxis(list(x_county_tv))
        .add_yaxis("first dose", y_first, yaxis_index=1, z=10)
        .add_yaxis("series complete", y_series, yaxis_index=1, z=10)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
bl1 = Bar1.overlap(Line1)

# "Top20 County fatality & ICU beds"
Bar2 = (
    Bar()
        .add_xaxis(list(x_county_dh))
        .add_yaxis("fatality", y_fata)
        .extend_axis(
            yaxis=opts.AxisOpts(
                type_='value', name='beds&ICUbeds', is_show=True, min_=0, max_=600, position='right'
            )
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="Top20 County Fatality & ICU beds"),
                        tooltip_opts=opts.TooltipOpts(trigger='axis',axis_pointer_type='cross'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
Line2 = (
    Line()
        .add_xaxis(list(x_county_dh))
        .add_yaxis("beds", y_bed, yaxis_index=1, z=10, 
                  )
        .add_yaxis("ICU beds", y_icu_bed, yaxis_index=1, z=10)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
bl2 = Bar2.overlap(Line2)

# "Top5 County Positive Trend"
Line3 = (
    Line()
        .add_xaxis(list(x1))
        .add_yaxis("Kings", y1)
        .add_yaxis("Queens", y2)
        .add_yaxis("New York", y3)
        .add_yaxis("Nassau", y4)
        .add_yaxis("Bronx", y5)
        .set_global_opts(title_opts=opts.TitleOpts(title="Top5 County Positive Trend"), 
                        tooltip_opts=opts.TooltipOpts(trigger='axis',axis_pointer_type='cross'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)

# "# of Foodstores And Positive Rate"
Bar3 = (
    Bar()
        .add_xaxis(list(x_county))
        .add_yaxis("# of foodstores", y_fs)
        .extend_axis(
            yaxis=opts.AxisOpts(
                type_='value', name='positive rate', is_show=True, min_=0, max_=0.07, position='right'
            )
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="# of Foodstores And Positive Rate"),
                        tooltip_opts=opts.TooltipOpts(trigger='axis',axis_pointer_type='cross'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
Line4 = (
    Line()
        .add_xaxis(list(x_county))
        .add_yaxis("positive rate", y_posi_rate, yaxis_index=1)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
bl3 = Bar3.overlap(Line4)

page = Page()
page.add(bl1, bl2, Line3, bl3)
page.render("/Users/apple/Documents/GWU-Class/sem3-database/final/covid19.html")

'/Users/apple/Documents/GWU-Class/sem3-database/final/covid19.html'